## Chapter 2 - Generating drum sequences with DrumsRNN

This example shows a basic Drums RNN generation with a hard coded primer.

1. Let's start by downloading the bundle:

In [1]:
import os

# There is a lot of useful tools in the magenta.music package,
# importing it as "mm" is pretty common in Magenta's code
import magenta.music as mm

mm.notebook_utils.download_bundle("drum_kit_rnn.mag", "bundles")
bundle = mm.sequence_generator_bundle.read_bundle_file(os.path.join("bundles", "drum_kit_rnn.mag"))

2. We then use the drums generator to initialize the generator class with the "drum_kit" configuration:

In [2]:
# Notice each model has its own package and generator
from magenta.models.drums_rnn import drums_rnn_sequence_generator

generator_map = drums_rnn_sequence_generator.get_generator_map()
generator = generator_map["drum_kit"](checkpoint=None, bundle=bundle)
generator.initialize()

3. This time, we'll define the number of bar we want to generate, instead of steps. By declaring the tempo, we can also calculate the length of a bar in seconds. We need this because the generation start and end is given in seconds to Magenta.

In [4]:
from magenta.music import constants

# We will generate 3 bars, so with a
# 1 bar primer we'll have 4 bars total
num_bars = 3
qpm = 120

# The steps per quarter for this generator
# is 4 steps per quarter
seconds_per_step = 60.0 / qpm / generator.steps_per_quarter

# We are using a default 16 steps per bar, which is
# 4/4 music sampled at 4 steps per quarter note
num_steps_per_bar = constants.DEFAULT_STEPS_PER_BAR

# We calculate how many seconds per bar for
# the generation time
seconds_per_bar = num_steps_per_bar * seconds_per_step

print("Seconds per step: " + str(seconds_per_step))
print("Seconds per bar: " + str(seconds_per_bar))

Seconds per step: 0.125
Seconds per bar: 2.0


4. We are now ready to initialize our primer sequence. We'll use a small jazz drum sequence of 1 bar for the primer (you can check it out in the book's source code in the Chapter 2 folder "primers/Jazz_Drum_Basic_1_bar.mid"), so we'll need a list of 16 step. We'll be explaining the primer definition in the next section. We can also initialize the start and end times for the generation of the next step:

In [6]:
# Define a 1 bar primer and converts it to a NoteSequence
primer_drums = mm.DrumTrack(
    [frozenset(pitches) for pitches in
        [(38, 51),     (), (36,),    (),
         (38, 44, 51), (), (36,),    (),
         (),           (), (38,),    (),
         (38, 44),     (), (36, 51), (),]])
primer_sequence = primer_drums.to_sequence(qpm=qpm)

# We store those time because the generation
# will start after the end of the primer
primer_start_time = 0
primer_end_time = primer_start_time + seconds_per_bar

# We calculate the generation start and end
# for a duration of num_bars
generation_start_time = primer_end_time
generation_end_time = generation_start_time + (seconds_per_bar * num_bars)

print("Primer start and end: [" + str(primer_start_time) + ", " + str(primer_end_time) + "]")
print("Generation start and end: [" + str(generation_start_time) + ", " + str(generation_end_time) + "]")

Primer start and end: [0, 2.0]
Generation start and end: [2.0, 8.0]


5. We can now configure our generator options with the start and end time. The generation options also takes the temperature:

In [8]:
# The generator interface is common for all models
generator_options = generator_pb2.GeneratorOptions()

# Add a bit of temperature for more flavor
temperature = 1.1
print("Temperature: " + str(temperature))
generator_options.args['temperature'].float_value = temperature

# Defines the generation section
generator_options.generate_sections.add(
    start_time=generation_start_time,
    end_time=generation_end_time)

Temperature: 1.1


start_time: 2.0
end_time: 8.0

6. It is time to generate! You can now call the generate method on the generator with the primer sequence as input:

In [9]:
# The resulting sequence is a NoteSequence instance
sequence = generator.generate(primer_sequence, generator_options)

7. There are many utilities to then convert a resulting NoteSequence to other formats such as PrettyMidi. We'll now convert the result, write the file and the plot on disk.

In [11]:
from visual_midi import Plotter

# Write the resulting midi file to the output directory
midi_file = os.path.join("output", "out.mid")
mm.midi_io.note_sequence_to_midi_file(sequence, midi_file)
print("Generated midi file: " + str(os.path.abspath(midi_file)))

# Write the resulting plot file to the output directory
plot_file = os.path.join("output", "out.html")
print("Generated plot file: " + str(os.path.abspath(plot_file)))
pretty_midi = mm.midi_io.note_sequence_to_pretty_midi(sequence)
plotter = Plotter()
plotter.show(pretty_midi, plot_file)

Generated midi file: C:/Users/Packt/hands-on-music-generation-with-magenta/Chapter02/output/out.mid
Generated plot file: C:/Users/Packt/hands-on-music-generation-with-magenta/Chapter02/output/out.html


Column(id='1101', ...)